<a href="https://colab.research.google.com/github/manthanawgan/MLops-Projects/blob/main/Nagpur_Temperature_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df = pd.read_csv('/content/historical-weather-dataset-ngp.csv')

df.head()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),sunshine_duration (s),shortwave_radiation_sum (MJ/m²),weather_code (wmo code),apparent_temperature_mean (°C),relative_humidity_2m_mean (%),dew_point_2m_mean (°C),cloud_cover_mean (%),wind_speed_10m_mean (km/h),surface_pressure_mean (hPa),et0_fao_evapotranspiration_sum (mm),soil_temperature_0_to_7cm_mean (°C),soil_moisture_0_to_10cm_mean (undefined)
0,1/1/2013,24.7,30.5,20.8,35383.75,13.64,61,27.8,78.0,20.3,26.0,6.2,975.5,2.98,25.8,NaN
1,1/2/2013,24.1,29.4,18.1,35423.55,13.53,2,26.7,74.0,19.0,18.0,6.2,976.3,3.03,25.4,NaN
2,1/3/2013,21.3,27.6,15.6,35615.22,17.20,0,20.9,57.0,11.9,0.0,7.8,978.1,3.79,23.6,NaN
3,1/4/2013,20.8,27.3,14.7,35729.32,17.47,0,20.6,56.0,11.4,0.0,6.0,976.8,3.62,23.0,NaN
4,1/5/2013,21.5,27.7,15.5,35696.16,15.66,0,21.4,57.0,12.3,6.0,6.8,974.7,3.52,23.3,NaN
